Trabalho desenvolvido pelos seguintes alunos:
Lucca Machado da Silva (19207083)
Bernardo Pandolfi Costa (19207646)
Heitor Henrique da Silva (19206261)

In [35]:
import numpy as np
import math

In [36]:
#A função fixos irá definir os termos nas bordas, os quais são dados no enunciado do problema.
def fixos(matrix):
    size = matrix.shape[0]
    for i in reversed(range(size)):
        v = abs((i/(size-1))-1)
        matrix[i][0] = round(-(v**2),3)
        matrix[i][size-1] = round(1-(v**2),3)
    
    for i in range(0, size):
        v = i*(1/(size-1))
        matrix[size-1][i] = round(v**2,3)
        matrix[0][i] = round((v**2) - 1,3)

#A função fixos_sem_arredondar faz o mesmo, só que sem arredondar os valores.
#Logo, a função fixos serve apenas para apresentar os valores de uma maneira mais legível.
def fixos_sem_arredondar(matrix):
    size = matrix.shape[0]
    for i in reversed(range(size)):
        v = abs((i/(size-1))-1)
        matrix[i][0] = -(v**2)
        matrix[i][size-1] = 1-(v**2)
    
    for i in range(0, size):
        v = i*(1/(size-1))
        matrix[size-1][i] = v**2
        matrix[0][i] = (v**2) - 1

In [37]:
#Criando a matrix para a solução, como o teste será com 40 sub intervalos, precisamos adicionar 41
#seguindo a fórmula n+1, onde n é o número de sub intervalos.
matrix = np.zeros((11,11))
#Matrix 2 será uma copia da matrix, será usada para inserir a solução exata e calcular o erro.
matrix2 = matrix.copy()

In [38]:
fixos(matrix)
matrix2 = matrix.copy()

In [39]:
#Exibindo como a matriz fica após calcular os valores nas bordas.
print(matrix) #depois

[[-1.   -0.96 -0.84 -0.64 -0.36  0.  ]
 [-0.64  0.    0.    0.    0.    0.36]
 [-0.36  0.    0.    0.    0.    0.64]
 [-0.16  0.    0.    0.    0.    0.84]
 [-0.04  0.    0.    0.    0.    0.96]
 [ 0.    0.04  0.16  0.36  0.64  1.  ]]


In [40]:
def solution(matrix):
    size = matrix.shape[0] #tamanho da matriz
    qtd_incognitas = (size-2)**2  #quantidade de incognitas

    equations = np.array([[]]) 
    #o array equations será responsável por armazenar as incognitas de cada equação do sistema linear.
    values = np.array([]) 
    #o array values será responsável por armazenar os valores (igualdade) de cada equação do sistema linear.

    #o indice 1 das equations tem igualdade com o indice 1 dos values, assim em diante
    
    matrix_aux = np.zeros((size, size)) #matriz auxiliar para guardar a nomenclatura de cada variável

    x = 1 #valor de inicio
    for i in reversed(range(1, size-1)):
        for j in range(1, size-1):
            matrix_aux[i][j] = x
            x += 1

    #Acima criamos outra matriz para servir de referencia, tendo somente a nomenclatura de cada variável.

    #agora, temos uma matriz com a nomenclatura de cada incognita, e a matriz com os valores do centro
    
    for i in reversed(range(1,size-1)):
        #utilizamos reversed range aqui pois nosso sistema de coordenadas é invertido,
        #matrizes no python começam com o termo [0][0] no canto superior esquerdo.
        #enquanto para nós, o canto superior esquerdo é [n][n]

        for j in range(1,size-1):
            x = matrix[i-1][j] #termo acima do central
            x2 = matrix[i][j+1] #termo a direita do central
            x3 = matrix[i+1][j] #termo abaixo do central
            x4 = matrix[i][j-1] #termo a esquerda do central

            a = np.zeros((1,qtd_incognitas))
            #na linha acima criamos uma equação do sistema linear com todos os coeficientes zerados.

            total = 0
            #a variável total será responsável por armazenar o somatório dos valores de igualdade.

            #no if a seguir iremos verificar quais dos termos da borda são constantes

            #a lógica de verificação se um termo é constante é a seguinte:
            #se o termo for constante, ele estará nas bordas da matriz, os valores das bordas serão
            # (i-1) == 0, (i+1) == size-1, (j-1) == 0, (j+1) == size-1
            #o primeiro termo nos indica que estamos na última linha de variáveis, ou seja, valores acima são constantes
            #o segundo termo nos indica que estamos na primeira linha de variáveis, ou seja, valores abaixo são constantes
            #o terceiro termo nos indica que estamos na primeira coluna de variáveis, ou seja, valores a esquerda são constantes
            #o quarto termo nos indica que estamos na última coluna de variáveis, ou seja, valores a direita são constantes

            #vale salientar que i-1 é o termo acima e i+1 é o termo abaixo, devido ao de que nosso eixo de referencia (X) é o inverso do eixo de referencia do python

            if(i-1 == 0): 
                total += x
            else:
                indice = matrix_aux[i-1][j] #esse indice é basicamente a nomenclatura da variável
                a[0][int(indice-1)] = -1 #agora inserimos a variável no sitema linear, como ela não é a variável central, ela é -1
            if(i+1 == size-1):
                total += x3
            else:
                indice = matrix_aux[i+1][j]
                a[0][int(indice-1)] = -1
        
            if(j-1 == 0):
                total += x4
            else:
                indice = matrix_aux[i][j-1]
                a[0][int(indice-1)] = -1   
            
            if(j+1 == size-1):
                total += x2
            else:
                indice = matrix_aux[i][j+1]
                a[0][int(indice-1)] = -1
            
            a[0][int(matrix_aux[i][j]-1)] = 4 #inserindo o termo central no sistema linear

            equations = np.append(equations, a) #adicionando a equação recém gerada no sistema linear

            values = np.append(values, total) #adicionando o valor da igualdade referente a equação gerada no sistema linear

    equations = np.reshape(equations,(qtd_incognitas, qtd_incognitas)) #estamos fazendo pois queremos uma matrix e não um array
    #originalmente o shape de equations era (qtd_incognitas**2), agora temos (qtd_incognitas, qtd_incognitas)

    x = np.linalg.solve(equations, values) #resolvendo o sistema linear
    x = np.reshape(x, (size-2, size-2)) #reorganizando a resposta em uma matrix

    return x

In [41]:
def exact(matrix2):
    size = matrix2.shape[0]
    for i in reversed(range(1,size-1)):
        if i == size-2:
            y = (1/int(size-1))
        else: 
            y = abs((i/(size-1))-1)
        for j in range(1, size-1):
            x = j/(size-1)
            matrix2[i][j] = (x**2 - y**2)



In [42]:
exact(matrix2)

In [43]:
#print(matrix2)

In [44]:
respo = solution(matrix)
respo = np.flip(respo, 0)
#além disso, ao resolver o sitema linear a matrix é invertida novamente, por isso precisamos invertela mais uma vez para podermos comparar a resposta mais facilmente.
print(respo)

[[-6.00000000e-01 -4.80000000e-01 -2.80000000e-01 -3.34244417e-17]
 [-3.20000000e-01 -2.00000000e-01 -1.68102355e-17  2.80000000e-01]
 [-1.20000000e-01  2.44533309e-17  2.00000000e-01  4.80000000e-01]
 [-3.46944695e-18  1.20000000e-01  3.20000000e-01  6.00000000e-01]]


In [45]:
size = matrix2.shape[0]
matrix2 = np.delete(matrix2, 0, 0)
matrix2 = np.delete(matrix2, size-2, 0)
matrix2 = np.delete(matrix2, 0, 1)
matrix2 = np.delete(matrix2, size-2, 1)
#acima, deletamos os valores fixos da matrix2, para que possamos comparar com a resposta do sistema linear calculada.
print(matrix2)

[[-0.6  -0.48 -0.28  0.  ]
 [-0.32 -0.2   0.    0.28]
 [-0.12  0.    0.2   0.48]
 [ 0.    0.12  0.32  0.6 ]]


In [46]:

erro = 0
sumrespo = 0
summatrix = 0

for i in range(len(respo)):
    for j in range(len(respo)):
        sumrespo = sumrespo + abs(respo[i][j])
        summatrix = summatrix + abs(matrix2[i][j])

erro = abs(sumrespo - summatrix)

print(erro)

0.0
